# Supported Backends

Since adcc does not implement a self-consistent field procedure or any integral computation routine it is only an addon to existing backends providing such functionality. Before starting an ADC calculation a Hartree-Fock reference first needs to be computed in a third-party code. For this purpose four codes are tightly integrated with adcc: [molsturm](https://molsturm.org), [psi4](https://psicode.org/), [PySCF](https://github.com/pyscf/pyscf), and [veloxchem](https://veloxchem.org/).

While other codes are supported out of the box, they can still be used if the required data can be stored inside a python dictionary or in an HDF5 file in the format expected by adcc, see the [last section of this notebook](#Starting-calculations-from-pregenerated-data). One should note that adding support for further host programs is rather simple, see https://adc-connect.org/q/hostprograms for details and feel free to contact us if you want to add support for another code.

For our discussion we will mostly focus on PySCF and Psi4, the codes for which integration and performance is best and supported featureset is most exhaustive.

## PySCF backend

[PySCF](http://pyscf.org/) is a python-based package for a large range of quantum-chemistry methods. It can be easily installed from [pypi](https://pypi.org/project/pyscf/) or [conda](https://anaconda.org/pyscf/pyscf), see the [detailed installation instructions](http://pyscf.org/pyscf/install.html). See their great [tutorial](http://pyscf.org/pyscf/tutorial.html) for details how to setup geometries and SCF calculations.

The water example from [00_Overview.ipynb](00_Overview.ipynb) would look like this using PySCF:

In [ ]:
# Note: PySCF is not installed on try.adc-connect.org, so this cell won't work there.
import adcc
from pyscf import gto, scf
adcc.set_n_threads(2)  # Reduce threads for binder VMs (see 03_Tweaks.ipynb)

# Run SCF in pyscf using a cc-pVTZ basis
mol = gto.M(
    atom='O 0 0 0;'
         'H 0 0 1.795239827225189;'
         'H 1.693194615993441 0 -0.599043184453037',
    basis='cc-pvtz',
    unit="Bohr",
)
mf = scf.HF(mol)
mf.kernel()

# Run ADC(2) for 10 states
state = adcc.adc2(mf, n_singlets=10)
print(state.describe())

Notice that aspects such as the one- and two-electron integrals, the molecular geometry and so on are passed to adcc automatically without any additional intervention from the user.

##### Strengths and weaknesses of the PySCF backend
- Restricted and unrestricted references supported
- Fastest AO to MO transformation of of the supported host programs, thus usually best performance
- Supports the generation of core holes (X-ray emission spectroscopy)
- Incomplete integration of adcc into PySCF itself: Not all features of PySCF can be used with adcc.

## Psi4 backend

[Psi4](http://www.psicode.org/) is a quantum-chemistry package with a broad range of features including a rich and powerful python frontend. It can be easily installed from [conda](https://anaconda.org/psi4/psi4), see their [detailed installation instructions](http://psicode.org/psi4manual/1.3.2/external.html). For getting started with the code a [tutorial](http://psicode.org/psi4manual/1.3.2/psiapi.html) is available. 

For completeness we show again the water example from [00_Overview.ipynb](00_Overview.ipynb):

In [ ]:
import psi4
import adcc
adcc.set_n_threads(2)  # Reduce threads for binder VMs (see 03_Tweaks.ipynb)

# Run SCF in Psi4 using a cc-pVTZ basis
mol = psi4.geometry("""
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037
    symmetry c1
    units au
""")
psi4.core.be_quiet()
psi4.set_options({'basis': "cc-pvtz", })
_, wfn = psi4.energy('SCF', return_wfn=True)

# Run ADC(2) for 10 states
state = adcc.adc2(wfn, n_singlets=10)
print(state.describe())

##### Strengths and weaknesses of the Psi4 backend
- Restricted and unrestricted references supported
- Slightly slower than the PySCF backend, since AO to MO transformation not yet properly parallelised.
- Most feature complete integration, including a direct integration of adcc into Psi4 (see next section).

## Using adcc within the Psi4 package

Not only can Psi4 SCF results be used to drive ADC calculations in adcc, but the integration of both packages is so close, that the reverse can be achieved as well. The next code example shows the same water calculation as above, but this time using only functions from Psi4.

In [ ]:
import psi4

mol = psi4.geometry("""
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037
    symmetry c1
    units au
""")
psi4.core.set_output_file("/tmp/psi4.out")
psi4.set_options({
    "basis": "cc-pvtz",
    "roots_per_irrep": [10],
    "kind": "singlet",
    "qc_module": "adcc",
})
psi4.properties("adc(2)", properties=["oscillator_strength"])

print(open("/tmp/psi4.out", "r").read())

In a similar way adcc calculations can be directly run from Psi4's [Psithon plain text input files](http://www.psicode.org/psi4manual/1.3.2/psithoninput.html) as well if desired.

## Other backends

This section presents the above water example for the other supported backends.

### molsturm
**Website:** https://molsturm.org 

In [ ]:
# Note: molsturm is not installed on try.adc-connect.org, so this cell won't work there.
import molsturm
import adcc

# Run Hartree-Fock in 
water = molsturm.System(["O", "H", "H"], [
    [0, 0, 0],
    [0, 0, 1.795239827225189],
    [1.693194615993441, 0, -0.599043184453037]
])
scfres = molsturm.hartree_fock(water, basis_type="gaussian", basis_set_name="cc-pvdz")

# Run ADC(2) for 10 states
state = adcc.adc2(scfres, n_singlets=10)
print(state.describe())

### Veloxchem
**Website:** https://veloxchem.org  
**Documentation:** https://veloxchem.org/manual

In [ ]:
# Note: veloxchem is not installed on try.adc-connect.org, so this cell won't work there.
import os
import adcc
import tempfile

import veloxchem as vlx
from mpi4py import MPI
from veloxchem.mpitask import MpiTask

# Run SCF in Veloxchem
with tempfile.TemporaryDirectory() as tmpdir:
    infile = os.path.join(tmpdir, "vlx.in")
    outfile = os.path.join(tmpdir, "vlx.out")

    with open(infile, "w") as fp:
        fp.write("""
                 @jobs
                 task: hf
                 @end

                 @method settings
                 basis: cc-pvtz
                 @end

                 @molecule
                 charge: 0
                 multiplicity: 1
                 units: bohr
                 xyz:
                 O 0 0 0
                 H 0 0 1.795239827225189
                 H 1.693194615993441 0 -0.599043184453037
                 @end
                 """)
    task = MpiTask([infile, outfile], MPI.COMM_WORLD)
    scfdrv = vlx.ScfRestrictedDriver(task.mpi_comm, task.ostream)
    scfdrv.compute(task.molecule, task.ao_basis, task.min_basis)
    scfdrv.task = task

# Run ADC(2) for 10 states
state = adcc.adc2(scfdrv, n_singlets=10)
print(state.describe())

## Starting calculations from pregenerated data

If adcc does not have an interface for the SCF code of your choice, you can still start an ADC calculation in adcc from the Hartree Fock results. The only requirement is that you can export the Fock matrix, the two-electron integrals and a few other quantities. The latter you can feed directly to adcc in a python dictionary or by reading them from an HDF5 file. For details on the format of the expected data, see https://adc-connect.org/q/hostprograms.

To provide you with a small example this repository is shipped with the HDF5 file `h2o_sto3g_hfdata.hdf5`, which contains precomputed data for water in an STO-3G basis in the expected format. To start a calculation from such data, just pass the file name of the HDF5 file or the python dictionary to adcc:

In [ ]:
import adcc
state = adcc.adc2x("h2o_sto3g_hfdata.hdf5", n_singlets=5)
print(state.describe())